In [2]:
from os import sep
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_absolute_error

df = pd.read_csv('FormattedData/train.csv', sep = ',' , header=0, parse_dates=['Timestamp'])

df.head()




2022-04-19 09:47:09.416466: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-19 09:47:09.416511: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


,Timestamp,MON1 CL DDM (?A),MON1 CL RF (dB),MON1 DS DDM (?A),MON1 DS RF (dB),MON1 NF DDM (?A),MON1 NF RF (dB),MON1 CLR DDM (?A),MON1 CLR RF (dB)
0,2022-01-28 14:11:19,2.02,0.32,0.87,0.29,-3.18,0.19,2.31,0.34
1,2022-01-28 14:16:19,1.74,0.32,0.87,0.29,-2.89,0.19,2.31,0.34
2,2022-01-28 14:21:20,1.74,0.32,0.87,0.29,-3.18,0.19,2.60,0.34
3,2022-01-28 14:26:20,0.29,-39.90,-76.94,-39.90,0.00,-39.90,-322.78,-35.61
4,2022-01-28 14:31:20,0.00,-39.90,-76.94,-39.90,0.00,-39.90,-322.49,-35.61


In [3]:
df.index = pd.to_datetime(df['Timestamp'].astype('datetime64').astype(int), format='%Y-%m-%d %H:%M:%S')

#Filling in null values
df = df.replace('?', np.nan)
df.isnull().sum()

#df.head()

Timestamp            0
MON1 CL DDM (?A)     0
MON1 CL RF (dB)      0
MON1 DS DDM (?A)     0
MON1 DS RF (dB)      0
MON1 NF DDM (?A)     0
MON1 NF RF (dB)      0
MON1 CLR DDM (?A)    0
MON1 CLR RF (dB)     0
dtype: int64

In [7]:
train_df, test_df = df[1:2529], df[2529:]

#Scale values from -1 to 1
train = train_df
scalers = {}

for i in train_df.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s = np.reshape(s_s, len(s_s))
    scalers['scaler_' + i] = scaler
    train[i] = s_s

test = test_df
for i in train_df.columns:
    scaler = scalers['scaler_' + i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s = np.reshape(s_s, len(s_s))
    scalers['scaler_' + i] = scaler
    test[i] =s_s


#Convert series to samples
def split_series (series, n_past, n_future):
    #
    #n_past => number of past observations
    #
    #n_future => number of future observations
    #
    #
    X, y = list(), list()
    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end >len(series):
            break
        
    # slicing the past and future parts of the window
        past, future = series[window_start:past_end, :], series[past_end:future_end, :]
        X.append(past)
        y.append(future)
    return np.array(X), np.array(y)

n_past = 10
n_future = 5
n_features = 9

X_train, y_train = split_series(train.values,n_past, n_future)
#X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
#y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
#X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
#y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))




/tmp/ipykernel_1166/1594298119.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i] = s_s
/tmp/ipykernel_1166/1594298119.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i] = s_s
/tmp/ipykernel_1166/1594298119.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [8]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])
#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)
#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)
#
#print(model_e1d1.summary())

# E2D2
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)
#
model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
#
model_e2d2.summary()

2022-04-19 09:54:10.810631: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-19 09:54:10.810679: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-19 09:54:10.810714: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-8JQCUIJ): /proc/driver/nvidia/version does not exist
2022-04-19 09:54:10.811077: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 10, 9)]      0           []                               
                                                                                                  
 lstm_2 (LSTM)                  [(None, 10, 100),    44000       ['input_2[0][0]']                
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                                  
 lstm_3 (LSTM)                  [(None, 100),        80400       ['lstm_2[0][0]']                 
                                 (None, 100),                                               

In [9]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])
#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)
#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)
#
#print(model_e1d1.summary())

# E2D2
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)
#
model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
#
model_e2d2.summary()


#Training models. Adam optimizer/Huber loss
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=0,callbacks=[reduce_lr])
model_e2d2.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2=model_e2d2.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=0,callbacks=[reduce_lr])


#Predict on test samples
pred_e1d1=model_e1d1.predict(X_test)
pred_e2d2=model_e2d2.predict(X_test)

pred_e1d1

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 10, 9)]      0           []                               
                                                                                                  
 lstm_8 (LSTM)                  [(None, 10, 100),    44000       ['input_4[0][0]']                
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                                  
 lstm_9 (LSTM)                  [(None, 100),        80400       ['lstm_8[0][0]']                 
                                 (None, 100),                                               

array([[[0.99231213, 0.29471144, 0.9818646 , ..., 0.9532724 ,
         0.9625031 , 0.97704405],
        [0.99359137, 0.29864204, 0.9772228 , ..., 0.9507662 ,
         0.95885265, 0.9723835 ],
        [1.0006138 , 0.29907253, 0.9783205 , ..., 0.95069075,
         0.95827246, 0.9673355 ],
        [0.9994216 , 0.30118072, 0.97635716, ..., 0.94889134,
         0.95379966, 0.9647023 ],
        [0.9934177 , 0.30420575, 0.97259474, ..., 0.94827837,
         0.9494043 , 0.965648  ]],

       [[0.99181443, 0.32322633, 0.9825298 , ..., 0.9547569 ,
         0.9645328 , 0.9773594 ],
        [0.993723  , 0.32738748, 0.97814554, ..., 0.95319355,
         0.9622385 , 0.97362494],
        [1.0008597 , 0.32685825, 0.9792493 , ..., 0.95330137,
         0.961871  , 0.9685934 ],
        [0.9997797 , 0.32769558, 0.9773335 , ..., 0.9515223 ,
         0.95738226, 0.9658291 ],
        [0.9938752 , 0.32941997, 0.97360677, ..., 0.9508355 ,
         0.9528861 , 0.9665986 ]],

       [[0.99200594, 0.341304  , 0.9

In [10]:
#Reverse scale values
for index,i in enumerate(train_df.columns):
    scaler = scalers['scaler_'+i]
    #pred1_e1d1[:,:,index]=scaler.inverse_transform(pred1_e1d1[:,:,index])
    pred_e1d1[:,:,index]=scaler.inverse_transform(pred_e1d1[:,:,index])
    #pred1_e2d2[:,:,index]=scaler.inverse_transform(pred1_e2d2[:,:,index])
    pred_e2d2[:,:,index]=scaler.inverse_transform(pred_e2d2[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])


for index,i in enumerate(train_df.columns):
  print(i)
  for j in range(1,6):
    print("Value",j,":")
    print("MAE-E1D1 : ",mean_absolute_error(y_test[:,j-1,index],pred_e1d1[:,j-1,index]),end=", ")
    print("MAE-E2D2 : ",mean_absolute_error(y_test[:,j-1,index],pred_e2d2[:,j-1,index]))

Timestamp
Value 1 :
MAE-E1D1 :  39803507265678.53, MAE-E2D2 :  35050410124771.863
Value 2 :
MAE-E1D1 :  35472446058219.91, MAE-E2D2 :  33677440209371.824
Value 3 :
MAE-E1D1 :  32122214290454.26, MAE-E2D2 :  33918050084673.547
Value 4 :
MAE-E1D1 :  33143469574184.504, MAE-E2D2 :  35917312993834.98
Value 5 :
MAE-E1D1 :  36495045471192.12, MAE-E2D2 :  39844539000312.27
MON1 CL DDM (?A)
Value 1 :
MAE-E1D1 :  0.15026561405347733, MAE-E2D2 :  0.19983444473017808
Value 2 :
MAE-E1D1 :  0.1756961705143324, MAE-E2D2 :  0.2060936359276517
Value 3 :
MAE-E1D1 :  0.1959744896865696, MAE-E2D2 :  0.19654568179218077
Value 4 :
MAE-E1D1 :  0.22023908787879384, MAE-E2D2 :  0.19186680786851515
Value 5 :
MAE-E1D1 :  0.24348439794807608, MAE-E2D2 :  0.19336182270648966
MON1 CL RF (dB)
Value 1 :
MAE-E1D1 :  0.5019971606266475, MAE-E2D2 :  0.29562392809489335
Value 2 :
MAE-E1D1 :  0.4260967318013035, MAE-E2D2 :  0.4337120356329671
Value 3 :
MAE-E1D1 :  0.3016477139925577, MAE-E2D2 :  0.444724702184393
Value 4